<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparation" data-toc-modified-id="Preparation-1">Preparation</a></span><ul class="toc-item"><li><span><a href="#Load-files" data-toc-modified-id="Load-files-1.1">Load files</a></span></li></ul></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span><ul class="toc-item"><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-2.1">Helper functions</a></span></li><li><span><a href="#Metric-functions" data-toc-modified-id="Metric-functions-2.2">Metric functions</a></span><ul class="toc-item"><li><span><a href="#Gaze-data" data-toc-modified-id="Gaze-data-2.2.1">Gaze data</a></span></li><li><span><a href="#Fixation-data" data-toc-modified-id="Fixation-data-2.2.2">Fixation data</a></span></li><li><span><a href="#Event-data" data-toc-modified-id="Event-data-2.2.3">Event data</a></span></li></ul></li></ul></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-3">Analysis</a></span><ul class="toc-item"><li><span><a href="#Baseline-Analysis" data-toc-modified-id="Baseline-Analysis-3.1">Baseline Analysis</a></span></li><li><span><a href="#FXD-Analysis" data-toc-modified-id="FXD-Analysis-3.2">FXD Analysis</a></span></li><li><span><a href="#EVD-Analysis" data-toc-modified-id="EVD-Analysis-3.3">EVD Analysis</a></span></li><li><span><a href="#GZD-Analysis" data-toc-modified-id="GZD-Analysis-3.4">GZD Analysis</a></span></li></ul></li></ul></div>

<h1>Eye Gaze Data Processing<span class="tocSkip"></span></h1>

Following metrics from: https://github.com/TheD2Lab/Eye.Tracking.Data.Analysis.For.Tobii.2150/tree/master/src/analysis

- Isaac Cortes
- Reynaldo Suarez
- Chris
- Janki

In [1]:
import pandas as pd
import numpy as np
import random 
import math

# Preparation

## Load files

In [2]:
def load_files(participant = 1):
    
    route = f'datasets/p{participant}/p{participant}'
    
    # Load one of the files to create functions
    baseline_cols = ['number', 'time', 'l_screen_x', 'l_screen_y', 'l_cam_x', 'l_cam_y', 'l_distance', 
                'l_pupil', 'l_code', 'r_screen_x', 'r_screen_y', 'r_cam_x', 'r_cam_y', 'r_distance', 
                'r_pupil', 'r_code']

    baseline = pd.read_csv(f'{route}GZD.txt', sep='\t', names = baseline_cols)
    
    # Load FXD graph and tree
    fxd_cols = ['number', 'time', 'duration', 'screen_x', 'screen_y']

    fxd_graph = pd.read_csv(f'{route}.graphFXD.txt', sep='\t', names = fxd_cols)
    fxd_tree = pd.read_csv(f'{route}.treeFXD.txt', sep='\t', names = fxd_cols)

    # Load EVD graph and tree
    evd_cols = ['time', 'event', 'event_key', 'data1', 'data2', 'description']

    evd_graph = pd.read_csv(f'{route}.graphEVD.txt', sep='\t', names = evd_cols)
    evd_tree = pd.read_csv(f'{route}.treeEVD.txt', sep='\t', names = evd_cols)
    
    # Load GZD graph and tree
    gzd_cols = ['number', 'time', 'l_screen_x', 'l_screen_y', 'l_cam_x', 'l_cam_y', 'l_distance', 
                'l_pupil', 'l_code', 'r_screen_x', 'r_screen_y', 'r_cam_x', 'r_cam_y', 'r_distance', 
                'r_pupil', 'r_code']

    gzd_graph = pd.read_csv(f'{route}.graphGZD.txt', sep='\t', names = gzd_cols)
    gzd_tree = pd.read_csv(f'{route}.treeGZD.txt', sep='\t', names = gzd_cols)

    return baseline, fxd_graph, fxd_tree, evd_graph, evd_tree, gzd_graph, gzd_tree


In [3]:
# TODO:
# Not all numbers 1-36 are participants in folder, use random choose instead with the folder contents
participant = random.randint(1,36)
print(f'Participant: {participant}')

baseline, fxd_graph, fxd_tree, evd_graph, evd_tree, gzd_graph, gzd_tree = load_files(participant)

Participant: 3


# Functions

## Helper functions

$$d = \sqrt{(x_2 - x_1)^2 + (y_2-y_1)^2}$$

In [4]:
def distance(row):
    x1, y1 = row['x'], row['y']
    x2, y2 =  row['next_x'], row['next_y']
    dist = math.sqrt(math.pow((x2-x1), 2) + math.pow((y2-y1), 2))
    row['dist'] = dist
    
    return row

$$dur = T2-(T1+D1)$$

In [5]:
def duration(row):
    
    t1, d1 = row['time'], row['duration']
    t2 = row['next_time']
    duration = t2 - (t1+d1)
    row['duration_between_fixations'] = duration
    
    return row 

## Metric functions

### Gaze data

In [6]:
def get_gaze_metrics(df):
    
    # A code with 0 indicates the eye tracker was confdident with this data
    # Filtering only records where both pupil sizes are valid
    df = df[(df['l_code'] == 0) & (df['r_code'] == 0)]
    avg_size_left = df["l_pupil"].mean()
    avg_size_right = df["r_pupil"].mean()
    avg_size_both = pd.concat([df["r_pupil"],df["l_pupil"]]).mean()
    
    return avg_size_left, avg_size_right, avg_size_both

In [7]:
def print_gaze_metrics(df):
    avg_size_left, avg_size_right, avg_size_both = get_gaze_metrics(df)

    print(f'Avg. pupil size of the left eye: {avg_size_left:.4f}')
    print(f'Avg. pupil size of the right eye: {avg_size_right:.4f}')
    print(f'Avg. pupil size of both eyes: {avg_size_both:.4f}')

### Fixation data

In [8]:
def get_fixation_metrics(df):
    
    total_fixations = len(df)
    sum_fixation_duration_sec = df["duration"].sum() / 1000
    mean_fixation_duration = df["duration"].mean()
    std_fixation_duration = df["duration"].std()
    
    return total_fixations, sum_fixation_duration_sec, mean_fixation_duration, std_fixation_duration
    
def get_saccade_length_metrics(df):
    
    # Get distances of all points
    coords = fxd_graph[['screen_x','screen_y']].copy()
    coords = coords.rename({'screen_x':'x','screen_y':'y'}, axis='columns')
    coords['next_x'] = coords['x'].shift(-1)
    coords['next_y'] = coords['y'].shift(-1)
    coords = coords.apply(distance, axis=1)
    coords.dropna(inplace=True)
    
    total_saccades = len(coords)
    sum_saccade_length = coords['dist'].sum()
    mean_saccade_length = coords['dist'].mean()
    std_saccade_length = coords['dist'].std()
        
    return total_saccades, sum_saccade_length, mean_saccade_length, std_saccade_length

def get_saccade_durations_metrics(df):
    
    # Get duration of all saccades
    saccadeDetails = fxd_graph[['time','duration']].copy()
    saccadeDetails['next_time'] = saccadeDetails['time'].shift(-1)
    saccadeDetails['next_duration'] = saccadeDetails['duration'].shift(-1)
    saccadeDetails = saccadeDetails.apply(duration, axis=1)
    saccadeDetails.dropna(inplace=True)
    
    sum_saccade_duration_sec = saccadeDetails['duration_between_fixations'].sum() / 1000
    mean_saccade_duration = saccadeDetails['duration_between_fixations'].mean()
    std_saccade_duration = saccadeDetails['duration_between_fixations'].std()
    
    return sum_saccade_duration_sec, mean_saccade_duration, std_saccade_duration

In [9]:
def print_fixation_metrics(df):
    # Fixations
    total_fixations, sum_fixation_duration_sec, mean_fixation_duration, std_fixation_duration = get_fixation_metrics(df)
    title = 'Fixations'
    print(f'{"="*10} {title:^20} {"="*10}')
    print(f'Total number of fixations: {total_fixations}')
    print(f'Sum of all fixation durations: {sum_fixation_duration_sec}s')
    print(f'Sum of all fixation durations: {sum_fixation_duration_sec / 60 :.2f}min')
    print(f'Mean fixation duration: {mean_fixation_duration:.2f}ms')
    print(f'StDev fixation duration: {std_fixation_duration:.2f}ms')

    # Saccade 
    total_saccades, sum_saccade_length, mean_saccade_length, std_saccade_length = get_saccade_length_metrics(df)
    title = 'Saccade lenghts'
    print(f'\n{"="*10} {title:^20} {"="*10}')
    print(f'Total number of saccades: {total_saccades}')
    print(f'Sum of all saccade lengths: {sum_saccade_length:.2f}px')
    print(f'Mean saccade length: {mean_saccade_length:.2f}px')
    print(f'StDev saccade length: {std_saccade_length:.2f}px')

    # Durations
    sum_saccade_duration_sec, mean_saccade_duration, std_saccade_duration = get_saccade_durations_metrics(df)
    title = 'Saccade durations'
    print(f'\n{"="*10} {title:^20} {"="*10}')
    print(f'Sum of all saccade durations: {sum_saccade_duration_sec:.2f}s')
    print(f'Sum of all saccade durations: {sum_saccade_duration_sec / 60 :.2f}min')
    print(f'Mean saccade duration: {mean_saccade_duration:.2f}ms')
    print(f'StDev saccade duration: {std_saccade_duration:.2f}ms')

### Event data

In [10]:
def get_event_metrics(df):
    
    lclicks = evd_graph[evd_graph['event'] == 'LMouseButton']
    lclicks = lclicks[['time','data1','data2']]
    lclicks['next_time'] = lclicks['time'].shift(-1)
    lclicks['time_between'] = lclicks['next_time'] - lclicks['time']
    lclicks.dropna(inplace=True)

    total_L_clicks = len(lclicks)
    avg_time_between_clicks_sec = lclicks['time_between'].mean() / 1000
    std_time_between_clicks_sec = lclicks['time_between'].std() / 1000
    
    return total_L_clicks, avg_time_between_clicks_sec, std_time_between_clicks_sec

In [11]:
def print_event_metrics(df):
    
    total_L_clicks, avg_time_between_clicks_sec, std_time_between_clicks_sec = get_event_metrics(df)
    
    print(f'Total number of L mouse clicks: {total_L_clicks}')
    print(f'Avg. amount of time between clicks: {avg_time_between_clicks_sec:.2f}s')
    print(f'StDev amount of time between clicks: {std_time_between_clicks_sec:.2f}s')

# Analysis

## Baseline Analysis

- average pupil size of left eye; 
- average pupil size of right eye; 
- average pupil size of both eyes.

In [12]:
baseline.head()

,number,time,l_screen_x,l_screen_y,l_cam_x,l_cam_y,l_distance,l_pupil,l_code,r_screen_x,r_screen_y,r_cam_x,r_cam_y,r_distance,r_pupil,r_code
0,19,1,-1280,-1024,0.695,0.541,-1.000,-1.000,4,-1280,-1024,0.462,0.564,-1.000,-1.000,4
1,39,2,694,502,0.694,0.541,706.735,3.223,0,703,493,0.461,0.565,708.977,3.253,0
2,59,3,682,484,0.694,0.541,706.735,3.153,0,684,487,0.462,0.565,708.977,3.267,0
3,79,4,487,501,0.697,0.541,706.735,3.185,0,450,474,0.465,0.564,708.977,3.336,0
4,99,5,450,489,0.698,0.541,706.735,3.188,0,444,488,0.466,0.564,708.977,3.296,0


In [13]:
print_gaze_metrics(baseline)

Avg. pupil size of the left eye: 2.9700
Avg. pupil size of the right eye: 3.0529
Avg. pupil size of both eyes: 3.0114


## FXD Analysis

**Fixations**
- total number of fixations
- sum of all fixation duration
- mean duration
- StDev of durations

**Saccade lengths**
- total number of saccades
- sum of all saccade length
- mean saccade length
- StDev of saccade lengths

**Saccade durations**
- sum of all saccade durations
- mean saccade duration
- StDev of saccade durations

**Research**
- scanpath duration
- fixation to saccade ratio

In [14]:
fxd_graph.head()

,number,time,duration,screen_x,screen_y
0,1,53,100,297,397
1,2,192,180,602,551
2,3,392,120,604,536
3,4,551,100,638,512
4,5,731,299,306,876


In [15]:
print_fixation_metrics(fxd_graph)

==========      Fixations       ==========
Total number of fixations: 7967
Sum of all fixation durations: 2462.609s
Sum of all fixation durations: 41.04min
Mean fixation duration: 309.10ms
StDev fixation duration: 249.35ms

==========   Saccade lenghts    ==========
Total number of saccades: 7966
Sum of all saccade lengths: 976608.46px
Mean saccade length: 122.60px
StDev saccade length: 160.17px

==========  Saccade durations   ==========
Sum of all saccade durations: 553.67s
Sum of all saccade durations: 9.23min
Mean saccade duration: 69.50ms
StDev saccade duration: 96.29ms


In [16]:
fxd_tree.head()

,number,time,duration,screen_x,screen_y
0,1,58,180,690,585
1,2,257,359,702,490
2,3,735,180,751,144
3,4,955,259,860,60
4,5,1234,319,547,251


In [17]:
print_fixation_metrics(fxd_tree)

==========      Fixations       ==========
Total number of fixations: 1564
Sum of all fixation durations: 515.632s
Sum of all fixation durations: 8.59min
Mean fixation duration: 329.69ms
StDev fixation duration: 275.84ms

==========   Saccade lenghts    ==========
Total number of saccades: 7966
Sum of all saccade lengths: 976608.46px
Mean saccade length: 122.60px
StDev saccade length: 160.17px

==========  Saccade durations   ==========
Sum of all saccade durations: 553.67s
Sum of all saccade durations: 9.23min
Mean saccade duration: 69.50ms
StDev saccade duration: 96.29ms


## EVD Analysis

- total number of L mouse clicks. 
- avg time between clicks.
- std time between clicks.

In [18]:
evd_graph.head()

,time,event,event_key,data1,data2,description
0,12566,LMouseButton,1,259,888,NaN
1,14579,LMouseButton,1,178,857,NaN
2,16642,LMouseButton,1,108,573,NaN
3,18675,LMouseButton,1,108,575,NaN
4,27753,LMouseButton,1,906,939,NaN


In [19]:
print_event_metrics(evd_graph)

Total number of L mouse clicks: 890
Avg. amount of time between clicks: 3.37s
StDev amount of time between clicks: 4.05s


In [20]:
evd_tree.head()

,time,event,event_key,data1,data2,description
0,810,LMouseButton,1,888,26,NaN
1,16159,LMouseButton,1,62,535,NaN
2,22997,LMouseButton,1,41,547,NaN
3,27053,LMouseButton,1,37,545,NaN
4,35136,LMouseButton,1,40,549,NaN


In [21]:
print_event_metrics(evd_tree)

Total number of L mouse clicks: 890
Avg. amount of time between clicks: 3.37s
StDev amount of time between clicks: 4.05s


## GZD Analysis

- average pupil size of left eye;
- average pupil size of right eye;
- average pupil size of both eyes.

In [22]:
gzd_graph.head()

,number,time,l_screen_x,l_screen_y,l_cam_x,l_cam_y,l_distance,l_pupil,l_code,r_screen_x,r_screen_y,r_cam_x,r_cam_y,r_distance,r_pupil,r_code
0,13,1,-1280,-1024,0.699,0.568,-1.000,-1.000,4,-1280,-1024,0.471,0.586,-1.00,-1.000,4
1,33,2,-1280,-1024,-1.000,-1.000,-1.000,-1.000,4,-1280,-1024,-1.000,-1.000,-1.00,-1.000,4
2,53,3,287,408,0.699,0.568,733.559,3.188,0,292,373,0.471,0.586,728.42,3.233,0
3,73,4,289,402,0.699,0.568,733.559,3.108,0,304,406,0.471,0.586,728.42,3.204,0
4,93,5,291,390,0.700,0.568,733.559,3.111,0,305,407,0.471,0.586,728.42,3.123,0


In [23]:
print_gaze_metrics(gzd_graph)

Avg. pupil size of the left eye: 3.1112
Avg. pupil size of the right eye: 3.1696
Avg. pupil size of both eyes: 3.1404


In [24]:
gzd_tree.head()

,number,time,l_screen_x,l_screen_y,l_cam_x,l_cam_y,l_distance,l_pupil,l_code,r_screen_x,r_screen_y,r_cam_x,r_cam_y,r_distance,r_pupil,r_code
0,18,1,-1280,-1024,0.689,0.607,-1.000,-1.000,4,-1280,-1024,0.462,0.628,-1.000,-1.000,4
1,38,2,-1280,-1024,-1.000,-1.000,-1.000,-1.000,4,-1280,-1024,-1.000,-1.000,-1.000,-1.000,4
2,58,3,690,598,0.689,0.607,730.478,3.196,0,702,587,0.463,0.628,729.819,3.262,0
3,78,4,670,571,0.690,0.607,730.478,3.223,0,700,587,0.463,0.628,729.819,3.266,0
4,98,5,675,592,0.690,0.607,730.478,3.202,0,699,586,0.463,0.628,729.819,3.243,0


In [25]:
print_gaze_metrics(gzd_tree)

Avg. pupil size of the left eye: 3.1564
Avg. pupil size of the right eye: 3.2201
Avg. pupil size of both eyes: 3.1883
